## Purpose

Export dataset from parquet to csv that I can use in Stata

I want: 1 row per person, with all the variables I need for my analysis

The variables are:

Outcome variables:
- `log_income`
- `log_wealth`
- `type_of_job_held`
- need to construct variables for movers and stayers

Control variables
- `age`
- `gender`
- `HISCLASS`
- `Schooling`
- `MaritalStatus`

Inclusion variables
- `distance_from_parish_to_line`
- `touching_treated_parish`

Variables for clustering
- `parish`

## So what do I need to do to construct these variables?

I need to join the first stage data to the birth place information

In [2]:
import pandas as pd
import pathlib
import numpy as np

root = pathlib.Path.cwd().parent

In [3]:
# read in census data
df_census_1930 = pd.read_parquet(root / "data/census/df_ref.parquet")
df_census_1930 = df_census_1930.rename(columns={'arkbild': 'name', 'scbkod': 'ref_code_current_parish', 'fscbkod': 'ref_code_birth_parish'})

In [4]:
# remove whitespace from ref_code_birth_parish
df_census_1930["ref_code_birth_parish"] = df_census_1930["ref_code_birth_parish"].str.strip()

# convert to int32, if it is not possible, it will be converted to NaN
df_census_1930["ref_code_birth_parish"] = pd.to_numeric(df_census_1930["ref_code_birth_parish"], errors="coerce")

# if df_census_1930["ref_code_current_parish"] is nan, fill it with df_census_1930["ref_code"]
df_census_1930["ref_code_birth_parish"] = df_census_1930["ref_code_birth_parish"].fillna(df_census_1930["ref_code_current_parish"])

In [6]:
# read in first stage data from "data/first-stage/parish-level-power-station-data-vf.xlsx"
first_stage_data = pd.read_excel(root / "data/first-stage/parish-level-power-station-data-vf.xlsx")

# keep columns ref_code, parish, area, geom_id, treated, touching_treated, touching_treated.1, distance_to_line
first_stage_data = first_stage_data[["ref_code", "parish", "area", "geom_id", "treated", "touching_treated", "touching_treated.1", "distance_to_line"]]

# drop duplicates
first_stage_data = first_stage_data.drop_duplicates()

# prepend birth_parish_ to each column name
first_stage_data.columns = ["birth_parish_" + col for col in first_stage_data.columns]

# generate ref_code_birth_parish is equal to birth_parish_ref_code and remove "SE/" from the new variable
first_stage_data["ref_code_birth_parish"] = first_stage_data["birth_parish_ref_code"].str.replace("SE/", "")

# replace missing values with 0 in ref_code_birth_parish
first_stage_data["ref_code_birth_parish"] = first_stage_data["ref_code_birth_parish"].fillna(0)

# remove the last three characters from ref_code_birth_parish
first_stage_data["ref_code_birth_parish"] = first_stage_data["ref_code_birth_parish"].astype(str).str[:-3]

# now convert ref_code_birth_parish to integer
first_stage_data["ref_code_birth_parish"] = first_stage_data["ref_code_birth_parish"].astype(int)

In [7]:
import numpy as np

# sort first_stage_data by ref_code_birth_parish
first_stage_data_sorted = first_stage_data.sort_values("ref_code_birth_parish")

# find closest match in first_stage_data for each value in df_census_1930
idx = first_stage_data_sorted["ref_code_birth_parish"].searchsorted(df_census_1930["ref_code_birth_parish"])

# create new column in df_census_1930 with closest match from first_stage_data
df_census_1930["closest_ref_code_birth_parish"] = first_stage_data_sorted.iloc[idx]["ref_code_birth_parish"].values

In [8]:

# merge df_census_1930 with first_stage_data on closest_ref_code_birth_parish
merged_data = pd.merge(df_census_1930, first_stage_data, left_on="closest_ref_code_birth_parish", right_on="ref_code_birth_parish", how="left")

In [9]:
df_census_1930 = merged_data

merged_data

,id,frnr,lannr,forsnr,pnr,abnr,name,kontrakt,ref_code_current_parish,lan,...,closest_ref_code_birth_parish,birth_parish_ref_code,birth_parish_parish,birth_parish_area,birth_parish_geom_id,birth_parish_treated,birth_parish_touching_treated,birth_parish_touching_treated.1,birth_parish_distance_to_line,ref_code_birth_parish_y
0,29170663,1930,1,1,1,133010027,Storkyrkoförsamlingen 1-4,Stockholms stad,18001,Stockholms stad,...,18101,SE/018101000,Södertälje församling,3.278788e+07,14866,0,0,0,123.903059,18101
1,29170664,1930,1,1,2,133010027,Storkyrkoförsamlingen 1-4,Stockholms stad,18001,Stockholms stad,...,18101,SE/018101000,Södertälje församling,3.278788e+07,14866,0,0,0,123.903059,18101
2,29170665,1930,1,1,3,133010027,Storkyrkoförsamlingen 1-4,Stockholms stad,18001,Stockholms stad,...,18001,SE/018001000,Stockholms domkyrkoförsamling,1.529956e+06,14093,0,0,0,129.443735,18001
3,29170666,1930,1,1,4,133010027,Storkyrkoförsamlingen 1-4,Stockholms stad,18001,Stockholms stad,...,18101,SE/018101000,Södertälje församling,3.278788e+07,14866,0,0,0,123.903059,18101
4,29170667,1930,1,1,5,133010027,Storkyrkoförsamlingen 1-4,Stockholms stad,18001,Stockholms stad,...,18101,SE/018101000,Södertälje församling,3.278788e+07,14866,0,0,0,123.903059,18101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338931,29052864,1930,25,32,2515,133250005,Haparanda,Norrb:s norra,258301,Norrbotten,...,258301,SE/258301000,Nedertorneå-Haparanda församling,4.731053e+08,12704,0,0,0,672.898060,258301
2338932,29052865,1930,25,32,2516,133250005,Haparanda,Norrb:s norra,258301,Norrbotten,...,258301,SE/258301000,Nedertorneå-Haparanda församling,4.731053e+08,12704,0,0,0,672.898060,258301
2338933,29052866,1930,25,32,2517,133250005,Haparanda,Norrb:s norra,258301,Norrbotten,...,258301,SE/258301000,Nedertorneå-Haparanda församling,4.731053e+08,12704,0,0,0,672.898060,258301
2338934,29052867,1930,25,32,2518,133250005,Haparanda,Norrb:s norra,258301,Norrbotten,...,258301,SE/258301000,Nedertorneå-Haparanda församling,4.731053e+08,12704,0,0,0,672.898060,258301


## Now join to current parish data

In [10]:
# read in first stage data from "data/first-stage/parish-level-power-station-data-vf.xlsx"
first_stage_data = pd.read_excel(root / "data/first-stage/parish-level-power-station-data-vf.xlsx")

# keep columns ref_code, parish, area, geom_id, treated, touching_treated, touching_treated.1, distance_to_line
first_stage_data = first_stage_data[["ref_code", "parish", "area", "geom_id", "treated", "touching_treated", "touching_treated.1", "distance_to_line"]]

# drop duplicates
first_stage_data = first_stage_data.drop_duplicates()

# prepend birth_parish_ to each column name
first_stage_data.columns = ["current_parish_" + col for col in first_stage_data.columns]

# generate ref_code_current_parish is equal to ref_code_current_parish and remove "SE/" from the new variable
first_stage_data["ref_code_current_parish"] = first_stage_data["current_parish_ref_code"].str.replace("SE/", "")
# replace missing values with 0 in ref_code_birth_parish
first_stage_data["ref_code_current_parish"] = first_stage_data["ref_code_current_parish"].fillna(0)

# remove the last three characters from ref_code_current_parish
first_stage_data["ref_code_current_parish"] = first_stage_data["ref_code_current_parish"].astype(str).str[:-3]

# now convert ref_code_current_parish to integer
first_stage_data["ref_code_current_parish"] = first_stage_data["ref_code_current_parish"].astype(int)

In [11]:
# sort first_stage_data by ref_code_current_parish
first_stage_data_sorted = first_stage_data.sort_values("ref_code_current_parish")

# find closest match in first_stage_data for each value in df_census_1930
idx = first_stage_data_sorted["ref_code_current_parish"].searchsorted(df_census_1930["ref_code_current_parish"])

# create new column in df_census_1930 with closest match from first_stage_data
df_census_1930["closest_ref_code_current_parish"] = first_stage_data_sorted.iloc[idx]["ref_code_current_parish"].values

# merge df_census_1930 with first_stage_data on closest_ref_code_birth_parish
merged_data = pd.merge(df_census_1930, first_stage_data, left_on="closest_ref_code_current_parish", right_on="ref_code_current_parish", how="left")

In [12]:
df_census_1930 = merged_data

In [13]:
df_census_1930.columns

Index(['id', 'frnr', 'lannr', 'forsnr', 'pnr', 'abnr', 'name', 'kontrakt',
       'ref_code_current_parish_x', 'lan',
       ...
       'closest_ref_code_current_parish', 'current_parish_ref_code',
       'current_parish_parish', 'current_parish_area',
       'current_parish_geom_id', 'current_parish_treated',
       'current_parish_touching_treated', 'current_parish_touching_treated.1',
       'current_parish_distance_to_line', 'ref_code_current_parish_y'],
      dtype='object', length=131)

In [14]:
# save a sample of 20 rows as excel file to check at "data/temp/census_dataset.xlsx"

df_census_1930.sample(20).to_excel(root / "data/temp/census_dataset.xlsx")

## Now choose the columns that we want to export to Stata

Outcome variables:
- `log_income`
- `log_wealth`
- `type_of_job_held`

Control variables
- `age`
- `gender`
- `HISCLASS`
- `Schooling`
- `MaritalStatus`
- 

Inclusion variables
- `distance_from_parish_to_line`
- `touching_treated_parish`

In [15]:
# log_income

df_census_1930['log_income'] = np.log(df_census_1930['income_incl_zero'] + 1)

In [16]:
# log wealth
df_census_1930['log_wealth'] = np.log(df_census_1930['formogh'] + 1)

In [17]:
# type of job held, direct electricity job

import pandas as pd
import numpy as np
import re

def direct_electricity_job(yrke_x):
    if re.search('elektr', yrke_x, re.IGNORECASE):
        return 1
    elif yrke_x == "Elmontör":
        return 1
    elif "Linjearbetare" in yrke_x:
        return 1
    elif "Kraftverksarbetare" in yrke_x:
        return 1
    else:
        return 0

v_direct_electricity_job = np.vectorize(direct_electricity_job)

df_census_1930['electricity_job_direct'] = v_direct_electricity_job(df_census_1930['yrke.x'])


In [18]:
def indirect_electricity_job(yrke_x):
    if yrke_x in [
      "Maskinist",
      "Pappersbruksarbetare",
      "Verkstadsarbetare",
      "Metallarbetare",
      "Textilarbeterska",
      "Sömmerska s. e.",
      "Träarbetare",
      "Järnverksarbetare",
      "Montör",
      "Valsverksarbetare",
      "Sömmerska s.e.",
      "Bryggeriarbetare",
      "Gjuteriarbetare",
      "Rörverksarbetare",
      "Smidesarbetare",
      "Pappersfabriksarbetare",
      "Maskinarbetare",
      "Plåtslageriarbetare",
      "Filare",
      "Skofabriksarbetare",
      "Järnbruksarbetare",
      "Slakteriarbetare",
      "Svarvare",
      "Glasslipare",
      "Tegelarbetare",
      "Snickerifabriksarbetare",
      "Färgeriarbetare",
      "Järnsvarvare",
      "Mejeribiträde",
      "Sågverksarbetare",
      "Tegelfabriksarbetare",
      "Mejerist",
      "Fabriksarbeterska",
      "Smedsarbetare",
      "Träsliperiarbetare",
      "Filare",
      "Sågare",
      "Mekaniker",
      "Tändsticksfabriksarbetare",
      "Telefonarbetare",
      "Skoarbetare",
      "Metallarbetare",
      "Snickeriarbetare",
      "Trämassearbetare",
      "Maskinsnickare",
      "Pappersbruksarbetare",
      "Textilarbetare Vävare",
      "Smedmästare",
      "Textilarbetare väv.",
      "Spinnerska",
      "Gjuteriarbetare",
      "Möbelsnickeriarbetare",
      "Tillskärare",
      "Pressare",
      "Mekanisk verkstadsarbetare",
      "Tandtekniker",
      "Varvsarbetare",
      "Martinarbetare",
      "Textilfabriksarbetare",
      "Järnvägsverkstadsarbetare",
      "Valsverksarbetare vid järnbruk",
      "Fabriksarbetare",
      "Smed s. e.",
      "Maskinist",
      "Sulfitfabriksarbetare",
      "Verkstadsarbetare",
      "Montör",
      "Tändsticksfabriksarbeterska",
      "Plåtverksarbetare",
      "Kopparslagare",
      "Lådfabriksarbetare",
      "Textilfabriksarbetare",
      "Gjutare",
      "Pappersarbetare",
      "Smedslärling",
      "Bilmontör",
      "Ingenjör",
      "Fabrikör",
      "Spolerska",
      "Textilfabriksarbeterska",
      "Hyttarbetare",
      "Smältverksarbetare",
      "Möbelfabriksarbetare",
      "Svarvare",
      "Tobaksarbetare",
      "Tråddragare",
      "Smed",
      "Konfektionssömmerska",
      "Pressare",
      "Bageriarbetare",
      "Slöjdare",
      "Kartongarbetare",
      "Verkmästare",
      "Sulfitfabriksarbetare",
      "Modellsnickare",
      "Plåtslagare",
      "Spinnare",
      "Järnsvarvare",
      "Ritare",
      "Tändsticksfabriksarbetare",
      "Skofabriksarbetare",
      "Motorskötare",
      "Appretörarbetare",
      "Chokladarbetare",
      "Maskinarbetare",
      "Trikåfabriksarbetare",
      "Cellulosafabriksarbetare",
      "Slaktare s.e.",
      "Bleckslagare",
      "Sockerbruksarbetare",
      "Telefonreparatör",
      "Garvare",
      "Linnesömmerska",
      "Trämassefabriksarbetare",
      "Martins-arbetare vid järnbruk",
      "Spikfabriksarbetare",
      "Spinneriarbetare",
      "Smidesarbetare",
      "Svetsare",
      "Korsettarbetare",
      "Yllefabriksarbetare",
      "Masugnsarbetare",
      "Smörjare",
      "Cellulosafabriksarbetare ",
      "Tryckeriarbetare",
      "Slipmassefabriksarbetare",
      "Reparatör",
      "Tegelbruksarbetare",
      "Bryggeriarbetare",
      "Smedarbetare",
      "Mekanikerarbetare",
      "Glasverksarbetare",
      "Textilarbetare Väverska",
      "Pälssömmerska",
      "Kalkbruksarbetare",
      "Bleckslageriarbetare",
      "Polerare",
      "Sågverksägare",
      "Faktor",
      "Stålverksarbetare",
      "Textilarbetare Rullerska",
      "Jutefabriksarbeterska",
      "Verktygsarbetare",
      "Rullerska",
      "Armaturarbetare",
      "Cementfabriksarbetare",
      "Hyvlare",
      "Litografiarbeterska",
      "Sågarbetare",
      "Fräsare",
      "Textilarbetare väveri",
      "Sågmästare",
      "Fabrikssömmerska",
      "Tegelarbetare",
      "Sortererska vid pappersbruk",
      "Tricotstickerska",
      "Laboratoriebiträde",
      "Textilarbetare Spinnare",
      "Fabriksarbetare papp.",
      "Stickerska s.e.",
      "Hemsömmerska",
      "Maskinpassare vid pappersbruk",
      "Varvsarbetare",
      "Rördrageriarbetare vid järnbruk",
      "Plåtslageriarbetare",
      "Emaljarbetare",
      "Tvinnerska",
      "Sågverksförman",
      "Sömmerska s. e.",
      "Trikåarbetare",
      "Kaolinbruksarbetare",
      "Fanerfabriksarbetare",
      "Lådfabriksarbetare",
      "Textilarbetare Sömmerska",
      "Textilarbetare Spolerska",
      "Maskinstickerska",
      "Fabriksarbetare cellulosafabrik",
      "Trämassefabriksarbetare",
      "Ritare",
      "Typograflärling",
      "Kappsömmerska",
      "Sömmerska textil",
      "Jutefabriksarbetare",
      "Fabriksarbeterska",
      "Fabriksarbetare sulfatfabrik",
      "Fabriksarbeterska Väverska",
      "Möbelfabrikör",
      "Klensmed",
      "Maskinförare vid pappersbruk",
      "Plåtslageriarbetare M. V.",
      "Tillskärerska",
      "Maskinskötare",
      "Smidesmästare",
      "Margarinfabriksarbetare",
      "Gummifabriksarbetare",
      "Maskinförare",
      "Skomakeriarbetare",
      "Avsynare",
      "Läderarbetare",
      "Karamellfabriksarbetare",
      "Plåtarbetare",
      "Metallduksvävare",
      "Tråddrageriarbetare",
      "Sömmerska Hemmadotter",
      "Snickeriförman",
      "Kappfabrikssömmerska",
      "Borrare",
      "Tryckare",
      "Borrare",
      "Sömmerska s.e.",
      "Snickerifabrikör",
      "Svarvare Arbetare",
      "Valsare",
      "Mejerinna",
      "Smärglare",
      "Smältare",
      "Brädgårdsförman",
      "Kakelfabriksarbetare",
      "Varperska",
      "Konservfabriksarbetare",
      "Mekaniker",
      "Svetsare",
      "Gjutmästare",
      "Järnsvarvare M. V.",
      "Kopparslageriarbetare",
      "Maskinsnickeriarbetare",
      "Karamellarbetare",
      "Kranmaskinist ",
      "Maskinmästare",
      "Smedsdräng",
      "Textilarbetare",
      "Snickare vid möbelfabrik",
      "Torvströfabriksarbetare",
      "Smedmästare ag.",
      "Bilmontör",
      "Guldlistarbetare",
      "Västsömmerska",
      "Sågverksarbetare f.",
      "Underofficer Maskinist",
      "Tråddrageriarbetare vid järnbruk",
      "Fabrikssnickare",
      "Tekniskt biträde",
      "Fabriksarbetare vid Kallvalsverk",
      "Spårvagnskonduktör",
      "Smedsgesäll",
      "Väveriarbetare",
      "Borstbinderiarbetare",
      "Textilförman",
      "Verkstadsförman",
      "Bobinfabriksarbetare",
      "Cykelarbetare",
      "Sågställare",
      "Plyserska",
      "Linderska",
      "Plåtslagare vid mekanisk verkstad",
      "Väverska textilfabrik",
      "Sorterare",
      "Galvaniseringsarbetare",
      "Industriarbetare",
      "Tricotsömmerska",
      "Beredningsarbetare",
      "Kopparslagare",
      "Väverska",
      "Maskinsättare",
      "Väverska textil",
      "Syfabrikssömmerska",
      "Lokomotivförare",
      "Radiotelegrafist",
      "Tunnfabriksarbetare",
      "Linjearbetare",
      "Plåtslagare s. e.",
      "Sågägare",
      "Sågverksmaskinist",
      "Maskinmjölkare",
      "Kranförare",
      "Maskinformare",
      "Telefonföreståndare",
      "Plåtslagare s.e.",
      "Barkhusarbetare Pappersbruk",
      "Eldare vid pappersbruk",
      "Textilarbetare varp.",
      "Kullagerarbetare",
      "Gravör",
      "Kärnmakare",
      "Väverska s. e.",
      "Sliparbetare",
      "Glödgare",
      "Lokförare järnväg",
      "Radiomontör",
      "Linslagare",
      "Diversearbetare Pappersbruk",
      "Sockerfabriksarbetare",
      "Porslinsfabriksarbetare",
      "Sömmerska Skräddare",
      "Tobaksarbeterska",
      "Stålsynare vid järnbruk",
      "Chokladarbeterska",
      "Facitarbetare",
      "Stabbläggare",
      "Mejerska",
      "Stenarbetareänka",
      "Tricotarbetare",
      "Silverpolererska",
      "Mejeribiträde",
      "Träindustriarbetare",
      "Maskinarbetare vid snickerifabrik",
      "Tapetfabriksarbetare",
      "Stenkrossarbetare",
      "Kartongfabriksarbetare",
      "Brukstjänsteman",
      "Glaspackare",
      "Möbelfabrikssnickare",
      "Fabriksarbetare Ägare",
      "Järnarbetare mekanisk verkstad",
      "Verkstadsarbetare Filare",
      "Gummireparatör",
      "Textilarbeterska",
      "Diamantborrare",
      "Rorgängare",
      "Textilarbetare kamgarnspinneri",
      "Bänkarbetare",
      "Cellulosafabriksarbetare Diversearbetare",
      "Sockerbruksarbetare",
      "Gjutare M. V.",
      "Övermaskinist",
      "Spiksmed",
      "Fräsare",
      "Gummiarbetare",
      "Margarinarbetare",
      "Klädesfabriksarbetare",
      "Bleckslageriarbetare",
      "Textilarbetare Appretör",
      "Ackumulatorsfabriksarbetare",
      "Lastare",
      "Bergsingenjör",
      "Porslinsarbetare",
      "Spinnare textil",
      "Urmakare",
      "Rensare",
      "Konfektionsarbetare",
      "Smärglare",
      "Repslageriarbetare",
      "Sömmerska Konfektion",
      "Vävare yllefabrik",
      "Sömnadsarbetare",
      "Borstarbetare",
      "Lagersömmerska",
      "Verkstadsarbetare vid järnbruk",
      "Verkstadsarbetare vid järnbruk",
      "Träullfabriksarbetare",
      "Järnarbetare Pappersbruk",
      "Synerska",
      "Konfektionspressare",
      "Stålvägare vid järnbruk",
      "Sömnadsarbeterska",
      "Glasarbetare",
      "aaaa",
      "Spolerska textil",
      "Mekanikerarbetare",
      "Glasskärare",
      "Rullerska textil",
      "Mek. arbetare",
      "Sågare",
      "Maskinreparatör",
      "Gödningsfabriksarbetare",
      "Glasslipare",
      "Reparatör vid pappersbruk",
      "Vävare textilfabrik",
      "Plåtslageriarbetare v. mek. verkst.",
      "Silverpolererska",
      "Tvålfabriksarbetare",
      "Gårdfarihandlare",
      "Sömmerska fabrik",
      "Biografmaskinist",
      "Textilarbetare Spolare",
      "Sömnadsarbetare",
      "Martinsarbetare",
      "Manglerska",
      "Litograf",
      "Sågbladsarbetare vid järnbruk",
      "Textilarbetare Tvisterska",
      "Byxsömmerska",
      "Väverska yllefabrik",
      "Hattfabriksarbetare",
      "Dussinsömmerska",
      "Kragsömmerska",
      "Cellulosafabriksarbetare Reparatör",
      "Lagerbiträde textil",
      "Litografarbetare",
      "Filhuggare",
      "Rullare vid pappersbruk",
      "Skofabriksarbeterska",
      "Fabriksarbetare cellulosa",
      "Putsare Arbetare",
      "Kantsågare",
      "Galvanisör",
      "Ingeniör",
      "Elektriker Montör",
      "Bindgarnsarbetare",
      "Bruksarbetare Gjutare",
      "Lackerare",
      "Sömmerska duss.",
      "Sättare",
      "Sömmerskearbetare",
      "Mejerimaskinist",
      "Varvsplåtslagare",
      "Filhuggare",
      "Stålsynare",
      "Hyttarbetare Järnverk",
      "Makaroniarbetare",
      "Magasinarbetare",
      "Pappersbrukfabriksarbetare",
      "Väskfabriksarbetare",
      "Textilarbetare Varperska",
      "Marmeladarbeterska",
      "Klädsömmerska",
      "Textilsömmerska",
      "Fabriksarbetare vid järnbruk",
      "Torkare vid pappersbruk",
      "Spinnerska textilfabrik",
      "Sömmerska se.",
      "Fabriksförman",
      "Utearbetare vid sulfitfabrik",
      "Textilfabriksarbeterska",
      "Tillskärare",
      "Mjölkning",
      "Tyglagerska yllefabrik",
      "Stickerska textil",
      "Typograf Sättare",
      "Textilarbetare Färgare",
      "Sintringsarbetare",
      "Påsfabriksarbetare",
      "Järnarbetare Kockum",
      "Sömmerska trikåfabrik",
      "Färgeriarbetare textil",
      "Konfektionsfabrikssömmerska",
      "Fabriksarbetare pappersbruk",
      "Sågförman",
      "Grovarbetare vid järnbruk",
      "Förtennare",
      "Vaddfabriksarbetare",
      "Tricotstickare",
      "Pressare konfektion",
      "Cellulosafabriksarbetare",
      "Presserska",
      "Instrumentmakare",
      "Reparatör Pappersbruk",
      "Segelsömmare",
      "Sömmerska vid fabrik",
      "Fabriksarbetare Eldare",
      "Torvfabriksarbetare",
      "Bindgarnsarbeterska",
      "Styckmästare",
      "Civilingenjör",
      "Vävare textil",
      "Stenslipare",
      "Tillskärare konfektion",
      "Maskinmontör",
      "Textilarbetare Solverska",
      "Maskinist vid sågverk",
      "Motorförare",
      "Hjälpmontör",
      "Väverska Holm. br.",
      "Revolversvarvare",
      "Varvsförman",
      "Sömmerska vid fabrik",
      "Järnarbetare vid pappersbruk",
      "Fabriksarbetare Eldare",
      "Gjuteriarbetare M. V.",
      "Torvfabriksarbetare",
      "Spisbrödsfabriksarbetare",
      "Bindgarnsarbeterska",
      "Sågverksarbetare Diversearbetare",
      "Civilingenjör",
      "Cementgjuteriarbetare",
      "Vävare textil",
      "Tillskärare konfektion",
      "Verkstadsägare",
      "Maskinmontör",
      "Textilarbetare Solverska",
      "Verksarbetare",
      "Maskinist vid sågverk",
      "Bryggeriägare",
      "Martins-arbetare",
      "Vällare",
      "Vattenfabriksarbetare",
      "Glasarbetare Slipare",
      "Järnarbetare Filare",
      "Gjuterihantlangare",
      "Manufakturarbetare",
      "Mejslare",
      "Trävaruarbetare",
      "Typograf Tryckare",
      "Telefonstationsföreståndarinna",
      "Sömmerska herrkonfektion",
      "Maskinförman",
      "Mejerist Föreståndare",
      "Resårfabriksarbetare",
      "Dessinatör",
      "Kärnmakare Arbetare",
      "Smed vid stenhuggeri",
      "Grovarbetare Pappersbruk",
      "Brädgårdsarbetare Stabbläggare",
      "Maskinslipare",
      "Sågverksarbetare Lägenhetsägare",
      "Snickeriarbetare Möbel",
      "Fabriksarbetare Pappersbruk",
      "Förman Pappersbruk",
      "Tändsticksfabriksförman",
      "Verkstadsarbetare vid mekanisk verkstad",
      "Bleckvarufabriksarbetare",
      "Linnearbetare",
      "Skräddarearbetare",
      "Ciselör",
      "Textilarbetare Diversearbetare",
      "Mejerist",
      "Filare mekanisk verkstad",
      "Mekanisk verkstadslärling",
      "Verkstadsarbetare Plåtslagare",
      "Järnsvarvare vid stålpressningsverk",
      "Sågverksarbetare Smörjare",
      "Telefonväxelföreståndare",
      "Filare vid stålpressningsverk",
      "Fabriksarbetare Slipare",
      "Rullare å pappersbruk",
      "Maskinsnickare vid möbelfabrik",
      "Textilarbetare Tvinnerska",
      "Kullagerarbetare",
      "Knivsmed",
      "Fräsare Arbetare",
      "Järnarbetare Kockums",
      "Sömmerska Lägenhetsägare",
      "Linnesömmerska",
      "Filare Mekanisk verkstad",
      "Sågverksarbetare Eldare",
      "Textilarbetare Spinneri",
      "Smed Arbetare",
      "Stenarbetare vid stenindustri",
      "Kontorist textil",
      "Radiotelegrafist",
      "Järnvägsverkstadsförman",
      "Typograf Maskinsättare",
      "Mekaniker s. e.",
      "Gjutmästare",
      "Vågfabriksarbetare",
      "Rörläggningsarbetare",
      "Rörläggare Arbetare",
      "Kabelarbetare",
      "Knivarbetare",
      "Verkmästare vid pappersbruk",
      "Textilarbetare Smörjare",
      "Verkstadsarbetare Svarvare",
      "Järnarbetare vid mekanisk verkstad",
      "Sågverksarbetare Chaufför",
      "Textilarbetare Holm. br.",
      "Arbetare å Kalmar Bobin",
      "Svarvarbetare",
      "Virapassare Pappersbruk",
      "Klensmed vid järnbruk",
      "Hjälprullare Pappersbruk",
      "Mekanisk verkstadsarbetare",
      "Textilarbetare bom. väv.",
      "Valsarbetare",
      "Stärkelsefabriksarbetare",
      "Martinarbetare Järnverk",
      "Radiofabriksarbetare",
      "Toffelfabriksarbetare",
      "Maskinuppsättare",
      "Möbelsnickarearbetare",
      "Svarvare vid bobinfabrik",
      "Riktare",
      "Gelbgjutare",
      "Metalltryckare",
      "Fabriksarbetare Grovarbetare",
      "Linjeförman",
      "Sodabrännare vid pappersbruk",
      "Träullsfabriksarbetare",
      "Sågverksarbetare Kantare",
      "Verkmästare vid mekanisk verkstad",
      "Sågarbetare vid chokladfabrik",
      "Tråddragare järnbruk",
      "Slipare vid pappersbruk",
      "Cellulosafabriksarbetare Eldare",
      "Mekaniker s.e."
    ]:
        return 1
    else:
        return 0

v_indirect_electricity_job = np.vectorize(indirect_electricity_job)

df_census_1930['electricity_job_indirect'] = v_indirect_electricity_job(df_census_1930['yrke.x'])


## Now choose the columns that we want to export to Stata

Control variables
- `age`
- `gender`
- `HISCLASS`
- `Schooling`
- `MaritalStatus`

Inclusion variables
- `distance_from_parish_to_line`
- `touching_treated_parish`

In [19]:
# age

df_census_1930['age']

0          72.0
1          48.0
2          41.0
3          24.0
4          23.0
           ... 
2672056     1.0
2672057    33.0
2672058    27.0
2672059    14.0
2672060    12.0
Name: age, Length: 2672061, dtype: float64

In [ ]:
# gender

df_census_1930["female"]

In [20]:
# marital status

def label_marital_status(status):
    if status == 'O':
        return 'Unmarried'
    elif status == 'G':
        return 'Married'
    elif status == 'E':
        return 'Widow/Widower'
    elif status == 'X':
        return 'Divorced'
    else:
        return 'Other'

v_label_marital_status = np.vectorize(label_marital_status)

df_census_1930['marital_status'] = v_label_marital_status(df_census_1930['civ'])


Create variable for age_2, employed and occ_title_without_income

In [21]:
# generate variable called age_2 which is age squared
df_census_1930['age_2'] = df_census_1930['age'] ** 2

# generate variable called employed = 1 if hisclass_group_abb is not missing, and zero otherwise
df_census_1930['employed'] = np.where(df_census_1930['hisclass_group_abb'].isnull(), 0, 1)

# generate a variable called occ_title_without_income which is eqaul is zero if hisclass_group_abb is not missing and log_income is not missing, is one if hisclass_group_abb is not missing and log_income is missing, and is missing if hisclass_group_abb is missing
# Set 'occ_title_without_income' to NaN by default
df_census_1930['occ_title_without_income'] = np.nan

# Set 'occ_title_without_income' to 0 when 'hisclass_group_abb' and 'log_income' are not NaN
df_census_1930.loc[(~df_census_1930['hisclass_group_abb'].isnull()) & (~df_census_1930['log_income'].isnull() & df_census_1930['log_income'] != 0), 'occ_title_without_income'] = 0

# Set 'occ_title_without_income' to 1 when 'hisclass_group_abb' is not NaN and 'log_income' is NaN or zero
df_census_1930.loc[(~df_census_1930['hisclass_group_abb'].isnull()) & (df_census_1930['log_income'].isnull() | df_census_1930['log_income'] == 0), 'occ_title_without_income'] = 1

In [22]:
# print all column names in df_census_1930
for col in df_census_1930.columns:
    print(col)


id
frnr
lannr
forsnr
pnr
abnr
name
kontrakt
ref_code_current_parish_x
lan
svarnr
kortnr
sida
rad
hemort
hnr
famst
fnr
famstkod
fornamn
enamn
titel
yrke.x
yrkkod
stam
lyte
fodar
fodort
fodfors
ref_code_birth_parish_x
kon
civ
trosb
utkyrk
utkyrkt
national
franvara
koppor
ejdop
status
klammer
struken
mantkod
rote
mantort
kyrkort
kallanm
ovrigt
anm
rplats
rsign
regdat
fodd
bildnr
cdnr
cdnamn
fodlan
fodlankod
xfornamn
xenamn
date_updated
hushtyp
kallref
hyrkstall
hyrkkod
hyrksoc
biyrke
biyrkkod
vigar
aupplar
inflar
inflfors
inflscb
skola
yrke25
yrkkod25
yrksoc25
barnfodd
barnfakt
barndoda
barndakt
barnhemv
inkomst_inmat
inkomst
formogh
formogh_inmat
bildid
ovf_obef_ar
age
adult_1930
lives_in_parish_of_birth
electricity_parish_living_in
electricity_parish_born
tce_group_100
tce_group_150
tce_group_250
dist_bp_to_cp_km
dist_pp_to_cp_km
yrke.y
occ_hisco
napp_title
hisco_code
hisclass_code
female
hisclass_code_abb
hisclass_group_abb
schooling
schooling_abb
income_imputed_median
income_imputed_m

### Add in the union density data

I did this in a file called "05_popula-movement-density.qmd" because I wanted to do it in R

The old way I did it is in python below but it was a mistake not to average across the 10 year windows.

In [24]:
import pandas as pd
import numpy as np

# Load the data from the second tab of the Excel file
data = pd.read_excel(root / "data/parishes/union_membership_and_treated_groups_merged.xlsx", sheet_name=1)

# Group by 'parish_code' and 'type_of_organization' and sum the number of members for 1900, 1910, and 1930
grouped_data = data.groupby(['parish_code', 'type_of_organization']).agg({
    'n_members_1900': 'sum',
    'n_members_1910': 'sum',
    'n_members_1930': 'sum',
    'population_1900': 'first',
    'population_1910': 'first',
    'population_1930': 'first'
}).reset_index()

# instead, group by 'parish_code' and 'type_of_organization' and take an average of a 10 year window around 1900, 1910, and 1930


# Calculate union density for 1900, 1910, and 1930
grouped_data['union_density_1900'] = (grouped_data['n_members_1900'] / grouped_data['population_1900']) * 100
grouped_data['union_density_1910'] = (grouped_data['n_members_1910'] / grouped_data['population_1910']) * 100
grouped_data['union_density_1930'] = (grouped_data['n_members_1930'] / grouped_data['population_1930']) * 100

# Joining the grouped_data with original data to get 'treated', 'distance_to_line', and 'touching_treated'
merged_data = pd.merge(grouped_data, data[['parish_code', 'treated', 'distance_to_line']], on='parish_code', how='left').drop_duplicates()

# Group parishes into 'treated', 'control', and 'other' based on the conditions
conditions = [
    (merged_data['treated'] == 1),
    (merged_data['treated'] == 0) & (merged_data['distance_to_line'] < 250)
]
choices = ['treated', 'control']
merged_data['group'] = np.select(conditions, choices, default='other')

# Cap the union densities at 100% for all years
merged_data['union_density_1900'] = np.clip(merged_data['union_density_1900'], 0, 100)
merged_data['union_density_1910'] = np.clip(merged_data['union_density_1910'], 0, 100)
merged_data['union_density_1930'] = np.clip(merged_data['union_density_1930'], 0, 100)

merged_data

# Pivot the dataframe
df_pivot = merged_data.pivot_table(index='parish_code', columns='type_of_organization', 
                          values=['union_density_1900', 'union_density_1910', 'union_density_1930'])

# Reset the index
df_pivot.reset_index(inplace=True)

# Flatten the MultiIndex columns
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]

# rename df_pivot to union_density
union_density = df_pivot

# replace NaN with zero for all variables
union_density = union_density.fillna(0)

# Define the column name mapping
column_mapping = {
    'union_density_1900_FACKF': 'popular_movement_density_1900_FACKF',
    'union_density_1900_FRIK': 'popular_movement_density_1900_FRIK',
    'union_density_1900_NYKT': 'popular_movement_density_1900_NYKT',
    'union_density_1900_PARTI': 'popular_movement_density_1900_PARTI',
    'union_density_1910_FACKF': 'popular_movement_density_1910_FACKF',
    'union_density_1910_FRIK': 'popular_movement_density_1910_FRIK',
    'union_density_1910_NYKT': 'popular_movement_density_1910_NYKT',
    'union_density_1910_PARTI': 'popular_movement_density_1910_PARTI',
    'union_density_1930_FACKF': 'popular_movement_density_1930_FACKF',
    'union_density_1930_FRIK': 'popular_movement_density_1930_FRIK',
    'union_density_1930_NYKT': 'popular_movement_density_1930_NYKT',
    'union_density_1930_PARTI': 'popular_movement_density_1930_PARTI'
}

# Rename the columns
union_density.rename(columns=column_mapping, inplace=True)

# save union density data to csv at root / "data/union_data/union_density.csv"
union_density.to_csv(root / "data/union-data/union_density.csv", index=False)

#### Join up to the census data

In [28]:
# Merge the dataframes
df_merged = df_census_1930.merge(union_density, how='left', 
                                 left_on='ref_code_current_parish_x', 
                                 right_on='parish_code_')

# Drop the 'parish_code' column
df_merged.drop('parish_code_', axis=1, inplace=True)

df_census_1930 = df_merged

## Export to parquet format

In [29]:
# export dataset to parquet file, in "data/census"
df_census_1930.to_parquet(root / "data/census/census_1930.parquet")

## Drop the unbalanced observations

In [5]:
control_parishes_dropped = pd.read_excel(root / "data/balance-tests/control_parishes_dropped.xlsx")

,parish_code,parish_code_short
0,11506000,11506
1,11704000,11704
2,12004000,12004
3,12005000,12005
4,12502000,12502
...,...,...
444,228210000,228210
445,228303000,228303
446,228307000,228307
447,230502000,230502


In [14]:
# read in df_census_1930 from parquet file
df_census_1930 = pd.read_parquet(root / "data/census/census_1930.parquet")

# how many rows and columns are in df_census_1930?
df_census_1930.shape

(2672061, 151)

In [15]:
# filter df_census_1930 to exclude parishes in control_parishes_dropped based on parish_code variable
df_census_1930 = df_census_1930[~df_census_1930['ref_code_current_parish_x'].isin(control_parishes_dropped['parish_code_short'])]

df_census_1930.to_parquet(root / "data/census/census_1930_excl_unbalanced_controls.parquet")

# How many rows and columns are in df_census_1930?
df_census_1930.shape

(1817340, 151)

### Export a sample to excel

In [ ]:
import pandas as pd

# Filter the dataframe
df_filtered = df_census_1930[(df_census_1930['birth_parish_distance_to_line'] <= 250) & 
                             (df_census_1930['age'].between(15, 100))]

# Get at least one row for each birth parish
df_sample = df_filtered.groupby('current_parish_parish').apply(lambda x: x.sample(1))

# Reset the index
df_sample.reset_index(drop=True, inplace=True)

# If the sample size is less than 5000, add more rows randomly
if len(df_sample) < 5000:
    remaining_rows = 5000 - len(df_sample)
    more_rows = df_filtered.loc[~df_filtered.index.isin(df_sample.index)].sample(remaining_rows)
    df_sample = pd.concat([df_sample, more_rows])

# Export to Excel
df_sample.to_excel(root / 'data/census/census_1930_sample_5000.xlsx', index=False)

## Export the census dataset to Stata format

In [16]:
# keep variables called log_income, log_wealth, electricity_job_direct, electricity_job_indirect, age, female, marital_status, hisclass_group_abb, schooling_abb, birth_parish_treated, current_parish_treated, birth_parish_distance_to_line, current_parish_distance_to_line, birth_parish_touching_treated, current_parish_touching_treated

df_census_1930 = df_census_1930.loc[:, ["id",
                                        "log_income",
                                        "log_wealth",
                                        "employed",
                                        "occ_title_without_income",
                                        "electricity_job_direct",
                                        "electricity_job_indirect",
                                        "popular_movement_density_1900_FACKF",
                                        "popular_movement_density_1910_FACKF",
                                        "popular_movement_density_1930_FACKF",
                                        "popular_movement_density_1900_FRIK",
                                        "popular_movement_density_1910_FRIK",
                                        "popular_movement_density_1930_FRIK",
                                        "popular_movement_density_1900_NYKT",
                                        "popular_movement_density_1910_NYKT",
                                        "popular_movement_density_1930_NYKT",
                                        "popular_movement_density_1900_PARTI",
                                        "popular_movement_density_1910_PARTI",
                                        "popular_movement_density_1930_PARTI",
                                        "age",
                                        "age_2", 
                                        "female", 
                                        "marital_status", 
                                        "hisclass_group_abb", 
                                        "schooling_abb", 
                                        "birth_parish_treated", 
                                        "current_parish_treated", 
                                        "birth_parish_distance_to_line", 
                                        "current_parish_distance_to_line", 
                                        "birth_parish_touching_treated", 
                                        "current_parish_touching_treated", 
                                        "birth_parish_ref_code", 
                                        "birth_parish_parish", 
                                        "current_parish_ref_code", 
                                        "current_parish_parish",
                                        "dist_bp_to_cp_km"]]

In [17]:
import os
directory = root / "data/census"

# export df_census_1930 to Stata format
filename = "1930_census_regression_dataset_excl_unbalanced_controls.dta"
filepath = os.path.join(directory, filename)
df_census_1930.to_stata(filepath)

/var/folders/xp/90b2vmmj0cg7rv097x887f6m0000gn/T/ipykernel_88691/679819266.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    popular_movement_density_1900_FACKF   ->   popular_movement_density_1900_FA
    popular_movement_density_1910_FACKF   ->   popular_movement_density_1910_FA
    popular_movement_density_1930_FACKF   ->   popular_movement_density_1930_FA
    popular_movement_density_1900_FRIK   ->   popular_movement_density_1900_FR
    popular_movement_density_1910_FRIK   ->   popular_movement_density_1910_FR
    popular_movement_density_1930_FRIK   ->   popular_movement_density_1930_FR
    popular_movement_density_1900_NYKT   ->   popular_movement_density_1900_NY
    popular_movement_density_1910_NYKT   ->   popular_movement_density_1910_NY
    popular_movement_density_1930_NYKT   ->   popular_movement_density_1930_NY
    popular_movement_density_1900_PARTI   ->   popular_movement_density_1900_PA


In [ ]:
df_census_1930

In [ ]:
# string detect "yrke" in col names df_census_1930?
# detect "yrke" in column names of df_census_1930
yrke_cols = [col for col in df_census_1930.columns if "yrke" in col]

if yrke_cols:
    print("Columns containing 'yrke':")
    print(yrke_cols)
else:
    print("No columns containing 'yrke' found.")